# Analysing ESG investing stocks by their scores

In [1]:
import pandas as pd
import yfinance as yf
import time
from random import randint
import warnings


In [2]:
warnings.filterwarnings('ignore')

### Tickers 

In [3]:
index_tickers= ['ADANIPORTS.NS',
'ASIANPAINT.NS',
'AXISBANK.NS',
"BAJAJ-AUTO.NS",
"BAJFINANCE.NS",
"BAJAJFINSV.NS"]

In [4]:
reliance = yf.Ticker("RELIANCE.NS")    #creating a new object for RELIANCE stock
reliance_df = reliance.sustainability.T   #getting ESG data from Yahoo in a dataframe format  
# pandas function .T is used to transpose the dataframe


In [5]:
esg_data = pd.DataFrame() #empty df for attaching all ticker's data response

for ticker in index_tickers:
     print(ticker) #just FYI to know your code is running
     ticker_name = yf.Ticker(ticker)
     try:
          if ticker_name.sustainability is not None: #if no response from Yahoo received, it will pass to next ticker
               ticker_df = ticker_name.sustainability.T #response dataframe
               ticker_df['symbol'] = ticker #adding new column 'symbol' in response df
               esg_data = esg_data.append(ticker_df) #attaching the response df to esg_data
               time.sleep(randint(2,8)) #delaying the fetch of data for 2-8 seconds
     except (IndexError, ValueError) as e: #in case yfinance API misbehaves
          print(f'{ticker} did not run') #FYI
          pass

ADANIPORTS.NS
ASIANPAINT.NS
AXISBANK.NS
BAJAJ-AUTO.NS
BAJFINANCE.NS
BAJAJFINSV.NS


### Data inside esg scorecards

In [6]:
esg_data.head(5) #calling first 5 rows

2022-8,palmOil,controversialWeapons,gambling,socialScore,nuclear,furLeather,alcoholic,gmo,catholic,socialPercentile,...,coal,pesticides,adult,percentile,peerGroup,smallArms,environmentScore,governancePercentile,militaryContract,symbol
Value,False,False,False,3.15,False,False,False,False,False,None,...,False,False,False,5.19,Transportation Infrastructure,False,3.61,None,False,ADANIPORTS.NS
Value,False,False,False,9.03,False,False,False,False,False,None,...,False,False,False,65.27,Chemicals,False,13.46,None,False,ASIANPAINT.NS
Value,False,False,False,12.07,False,False,False,False,False,None,...,False,False,False,48.69,Banks,False,1.94,None,False,AXISBANK.NS
Value,False,False,False,5.58,False,False,False,False,False,None,...,False,False,False,13.61,Automobiles,False,3.38,None,False,BAJAJ-AUTO.NS
Value,False,False,False,11.44,False,False,False,False,False,None,...,False,False,False,52.92,Insurance,False,2.27,None,False,BAJAJFINSV.NS


In [7]:
esg_tickers = esg_data['symbol']
no_esg_data = list(set(index_tickers) - set(esg_tickers))
#set function removes all duplicates in a list and the above gives us the
#difference between our original ticker list and our esg_data ticker list
print(no_esg_data)

['BAJFINANCE.NS']


In [8]:
new_esg_df = esg_data[['symbol', 'socialScore', 
               'governanceScore', 'totalEsg', 'environmentScore']]
#the above basically takes the columns mentioned above and assigns into new df.
new_esg_df.head(5) #let's see what it looks like

2022-8,symbol,socialScore,governanceScore,totalEsg,environmentScore
Value,ADANIPORTS.NS,3.15,5.62,12.37,3.61
Value,ASIANPAINT.NS,9.03,8.07,30.56,13.46
Value,AXISBANK.NS,12.07,12.13,26.15,1.94
Value,BAJAJ-AUTO.NS,5.58,7.35,16.31,3.38
Value,BAJAJFINSV.NS,11.44,13.59,27.31,2.27


### Sample data log

In [9]:
print(reliance.info)

{'ebitdaMargins': None, 'profitMargins': None, 'grossMargins': None, 'operatingCashflow': None, 'revenueGrowth': None, 'operatingMargins': None, 'ebitda': None, 'targetLowPrice': 1350, 'recommendationKey': 'buy', 'grossProfits': 1722910000000, 'freeCashflow': None, 'targetMedianPrice': 2280, 'currentPrice': 2552, 'earningsGrowth': None, 'currentRatio': None, 'returnOnAssets': None, 'numberOfAnalystOpinions': 29, 'targetMeanPrice': 2272.1, 'debtToEquity': None, 'returnOnEquity': None, 'targetHighPrice': 2830, 'totalCash': None, 'totalDebt': None, 'totalRevenue': None, 'totalCashPerShare': None, 'financialCurrency': 'INR', 'maxAge': 1, 'revenuePerShare': None, 'quickRatio': None, 'recommendationMean': 2.1, 'exchange': 'NSI', 'shortName': 'RELIANCE INDS', 'longName': 'Reliance Industries Limited', 'exchangeTimezoneName': 'Asia/Kolkata', 'exchangeTimezoneShortName': 'IST', 'isEsgPopulated': False, 'gmtOffSetMilliseconds': '19800000', 'quoteType': 'EQUITY', 'symbol': 'RELIANCE.NS', 'message

In [10]:
main_df = pd.DataFrame() #creating empty df to store data

for ticker in index_tickers:
     ticker_name = yf.Ticker(ticker)
     try:
          ticker_info = ticker_name.info
          ticker_df = pd.DataFrame.from_dict(ticker_info.items()).T
          #the above line will parse the dict response into a DataFrame
          ticker_df.columns = ticker_df.iloc[0]
          #above line will rename all columns to first row of dataframe
          #as all the headers come up in the 1st row, next line will drop the 1st line
          ticker_df = ticker_df.drop(ticker_df.index[0])
          main_df = main_df.append(ticker_df)
          time.sleep(randint(2,8))
          print(f'{ticker} + Complete')
     except (IndexError, ValueError) as e:
          print(f'{ticker} + Data Not Found')

ADANIPORTS.NS + Complete
ASIANPAINT.NS + Complete
AXISBANK.NS + Complete
BAJAJ-AUTO.NS + Complete
BAJFINANCE.NS + Complete
BAJAJFINSV.NS + Complete


### Sector wise

In [11]:
filtered_df = main_df[['symbol', 'sector', 'previousClose', 'sharesOutstanding']]
filtered_df.head(5) #checking how first 5 rows look like

,symbol,sector,previousClose,sharesOutstanding
1,ADANIPORTS.NS,Industrials,820.3,2160140032
1,ASIANPAINT.NS,Basic Materials,3053.4,959198016
1,AXISBANK.NS,Financial Services,903.05,3072430080
1,BAJAJ-AUTO.NS,Consumer Cyclical,3670.7,282956992
1,BAJFINANCE.NS,Financial Services,7006.85,605428992


In [12]:
filtered_df['newMarketCap'] = filtered_df['previousClose'] * filtered_df['sharesOutstanding']

total_index_mcap = filtered_df['newMarketCap'].sum()

filtered_df['marketWeight'] = ((filtered_df['newMarketCap']/total_index_mcap)*100)

In [13]:
filtered_df.head()

,symbol,sector,previousClose,sharesOutstanding,newMarketCap,marketWeight
1,ADANIPORTS.NS,Industrials,820.3,2160140032,1771962868249.599854,11.51395
1,ASIANPAINT.NS,Basic Materials,3053.4,959198016,2928815222054.399902,19.031003
1,AXISBANK.NS,Financial Services,903.05,3072430080,2774557983744.0,18.028663
1,BAJAJ-AUTO.NS,Consumer Cyclical,3670.7,282956992,1038650230534.399902,6.748994
1,BAJFINANCE.NS,Financial Services,7006.85,605428992,4242150132595.200195,27.564857


In [14]:
final_df = filtered_df.merge(new_esg_df, how='left', on='symbol')

In [15]:
final_df.head()

,symbol,sector,previousClose,sharesOutstanding,newMarketCap,marketWeight,socialScore,governanceScore,totalEsg,environmentScore
0,ADANIPORTS.NS,Industrials,820.3,2160140032,1771962868249.599854,11.51395,3.15,5.62,12.37,3.61
1,ASIANPAINT.NS,Basic Materials,3053.4,959198016,2928815222054.399902,19.031003,9.03,8.07,30.56,13.46
2,AXISBANK.NS,Financial Services,903.05,3072430080,2774557983744.0,18.028663,12.07,12.13,26.15,1.94
3,BAJAJ-AUTO.NS,Consumer Cyclical,3670.7,282956992,1038650230534.399902,6.748994,5.58,7.35,16.31,3.38
4,BAJFINANCE.NS,Financial Services,7006.85,605428992,4242150132595.200195,27.564857,NaN,NaN,NaN,NaN


In [16]:
final_df[~final_df['totalEsg'].notnull()]

,symbol,sector,previousClose,sharesOutstanding,newMarketCap,marketWeight,socialScore,governanceScore,totalEsg,environmentScore
4,BAJFINANCE.NS,Financial Services,7006.85,605428992,4242150132595.200195,27.564857,NaN,NaN,NaN,NaN


In [17]:
final_esg_df = pd.DataFrame() #empty df

sector_list = final_df['sector'].unique().tolist() #getting list of sectors in index

#looping over each sector and apply .mean to calculate average
for sector in sector_list:
    sector_df = final_df[final_df['sector'] == sector]
    sector_df['socialScore'].fillna(round(sector_df['socialScore'].mean(),2), inplace=True)
    sector_df['governanceScore'].fillna(round(sector_df['governanceScore'].mean(),2), inplace=True)
    sector_df['totalEsg'].fillna(round(sector_df['totalEsg'].mean(),2), inplace=True)
    sector_df['environmentScore'].fillna(round(sector_df['environmentScore'].mean(),2), inplace=True)

    final_esg_df = final_esg_df.append(sector_df)

#also adding the weighted average columns into this new final_esg_df
final_esg_df['mktweightedEsg'] = (final_esg_df['marketWeight'] * final_esg_df['totalEsg'])/100
final_esg_df['mktweightedEnvScore'] = (final_esg_df['marketWeight'] * final_esg_df['environmentScore'])/100
final_esg_df['mktweightedSocScore'] = (final_esg_df['marketWeight'] * final_esg_df['socialScore'])/100
final_esg_df['mktweightedGovScore'] = (final_esg_df['marketWeight'] * final_esg_df['governanceScore'])/100

## Final Scores

In [18]:
print('Total Environment Score: {}'.format(round(final_esg_df['mktweightedEnvScore'].sum(),2)))
print('Total Social Score: {}'.format(round(final_esg_df['mktweightedSocScore'].sum(),2)))
print('Total Governance Score: {}'.format(round(final_esg_df['mktweightedGovScore'].sum(),2)))
print('Total ESG Score: {}'.format(round(final_esg_df['mktweightedEsg'].sum(),2)))

Total Environment Score: 4.52
Total Social Score: 9.83
Total Governance Score: 10.74
Total ESG Score: 25.1
